<img align="left" src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/CC_BY.png"><br />

Created by Zhuo Chen for [Constellate](https://constellate.org) under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/).<br />
For questions/comments/improvements, email zhuo.chen@ithaka.org.<br />

# Data Visualization 3

This is lesson 3 of 3 in the educational series on **Data Visualization**. 

**Description:** This notebook discusses:

* Plot several subplots in a single figure
* Create histograms 
* Create an interactive figure

**Use case:** For Learners (Detailed explanation, not ideal for researchers)

**Difficulty:** Intermediate

**Completion time:** 90 minutes

**Knowledge Required:** 

* Python basics ([start learning Python basics](../Python-basics/python-basics-1.ipynb))
* Pandas ([start learning Pandas](../Pandas-basics/pandas-basics-1.ipynb))

**Knowledge Recommended:**

Objects, attributes and methods (start learning [object-oriented programming](../Python-intermediate/python-intermediate-4.ipynb))

**Data Format**: .csv

**Libraries Used**: Pandas, Matplotlib

**Research Pipeline**: None

___

## Import Required Libraries

In [ ]:
# import pyplot module
import matplotlib.pyplot as plt

# import pandas
import pandas as pd

# import ipympl
import ipympl

In [ ]:
# set the backend to matplotlib widget
%matplotlib widget

## Subplots

In [DataViz 1](./data-viz1.ipynb) and [DataViz 2](./data-viz2.ipynb), we only used a single axes object to plot a single graph. However in real life, there are times when it is useful to show several subplots in one figure so that the readers can compare.

When we create a figure, we can specify how many subplots we would like to create in the figure. 

Each subplot is contained in an axes object. The axes objects are organized in rows and columns. Therefore, we can specify the number of rows and columns to create the desired number of subplots.

In [ ]:
# create a figure with 4 subplots
fig, ax = plt.subplots(2, 2)

In [ ]:
# print out ax
print(ax)

The axes objects in the same row are put in one list. Therefore, when we print out the value stored in the variable `ax`, we get a list of lists.

If you just want a list that contains all the axes objects, you can flatten the list using `ax.flat`. 

It is easier for us to grab a certain subplot and do some operations to it if we assign the subplots to different variables when we create them. 

In [ ]:
# Assign the subplots to different variables
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)

In [ ]:
# put different artists on the four subplots
ax1.plot(1, 1, 'o')
ax2.plot([1,2], [4,3], '--')
ax3.scatter([1,2,3], [2,4,6], edgecolor='black', linewidth=1)
ax4.bar([1,2],[3,4], color='grey')
fig

Let's return to the childhood lead poisoning data for all 14 counties in MA. Suppose we would want to plot a figure that contains 14 line graphs, each for one of the 14 counties showing the lead poisoning data from 2012 - 2020. 

In [ ]:
# Get the path to the file
file_path ='../All-sample-files/DataViz3_ChildhoodLeadPoisoning_County20122020.csv'

Next, we create a dataframe from the csv file. 

In [ ]:
# Create a dataframe from the csv file
lead = pd.read_csv(file_path)
lead

Let's focus on the county names, years, and rate of children per 1,000 who had elevated blood lead level. We can reduce the dataframe to these three columns. Moreover, since we are not interested in the data broken down by the sex of the children, the only rows that we will maintain are the rows where the column `Sex` says `Total`.

In [ ]:
# Reduce the dataframe to the data of interest
lead_perc = lead.loc[lead['Sex']=='Total', ['Geo Description', 'Year', 'Rate']].copy().reset_index(drop=True)

# Change the rate column to percentage
lead_perc['Rate'] = lead_perc['Rate'] * 0.1

# Rename the columns to make them more descriptive
lead_perc = lead_perc.rename(columns={'Geo Description':'County', 'Rate':'Percentage'})

We are ready to plot the figure! 

In [ ]:
# import ticker for later use
import matplotlib.ticker as mtick

# create a figure with 7 rows and 2 columns
fig, axs = plt.subplots(nrows=7, ncols=2, figsize = (7, 15))

for county, ax in zip(lead_perc['County'].unique(), axs.flat):
    ax.plot(range(2012, 2021), lead_perc.loc[lead_perc['County']==county, 'Percentage'])
    ax.set_title(county, fontsize=9)
    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
fig.suptitle('Percentage of Children with Elevated BLL', y=0.99)
fig.supxlabel('Year')
fig.tight_layout()


You may have noticed that all the subplots share the same x-axis. We learned in [DataViz 1](./data-viz1.ipynb) that a good data visualization avoids unnecessary "chartjunk." We can simplify the figure by making all the subplots share the x-axis. That way, we don't need to repeat the same x-axis for every subplot.

There is a parameter `sharex` in the `subplots()` method that we can use to achieve this goal. 

If we doule like to share y-axis, we can set the parameter `sharey` to `True` as well. 

In [ ]:
# import ticker for later use
import matplotlib.ticker as mtick

# create a figure with 7 rows and 2 columns
fig, axs = plt.subplots(nrows=7, ncols=2, figsize = (7, 15), sharex=True, sharey=True)

# Get the counties
counties = lead_perc['County'].unique().tolist()

# Make the subplots
for i, ax in enumerate(axs.flat):
    ax.plot(range(2012, 2021), lead_perc.loc[lead_perc['County']==counties[i], 'Percentage'])
    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
    ax.set_title(counties[i], fontsize=9)
fig.suptitle('Percentage of Children with Elevated BLL', y=0.99)
fig.supxlabel('Year')
fig.tight_layout()

<h3 style="color:red; display:inline">Coding Challenge! &lt; / &gt; </h3>

In the next code cell, make a figure containing 14 subplots, each for one county of MA. In each subplot, make a line graph with two lines, one showing the percentage of the female children with elevated BLL from 2012 - 2020 and one showing the percentage of the male children with elevated BLL from 2012 - 2020. 

In [ ]:
### Make a figure containing multiple subplots
### showing the percentage of male and female children with elevated BLL by county


## Histograms

A histogram is a bar chart which shows the frequency of observations. In a histogram, the x-axis is a continuous quantitative value. The height of each bar shows the frequency of a certain range of values. The biggest difference between a bar chart and a histogram is that a bar chart has categorical values on the x-axis but a histogram has continuous quantitative values on the x-axis. 

In this section, let's use data on the 2022 Boston Marathon to plot a histogram. 

In [ ]:
# Get the path to the file
file ='../All-sample-files/DataViz3_BostonMarathon2022.csv'

In [ ]:
# Create a dataframe from the csv file
run = pd.read_csv(file)
run

There are 24,834 rows in this dataframe. That means there were 24,834 runners in the event last year! Suppose we are interested in the distribution of the age of the participants in ranges of 10. How many runners were of age 20 - 29 at the time of race? How many were of age 30 - 39?

In [ ]:
# Reduce the dataframe to the column of interest
run_age = run[['AgeOnRaceDay']].copy()
run_age = run_age.rename(columns={'AgeOnRaceDay':'Age'})
run_age

In [ ]:
# Get the max and min value of age to decide on bins
print(run_age['Age'].max())
print(run_age['Age'].min())

Take a guess which age group has the most runners? 20s? 30s? Let's find out by plotting the histogram.

In [ ]:
# Make a list of bins
bins = range(10, 100, 10)

# Plot the histogram
fig, ax = plt.subplots()
counts, edges, bars = ax.hist(run_age['Age'], bins = bins, edgecolor='black')
ax.bar_label(bars);

<h3 style="color:red; display:inline">Coding Challenge! &lt; / &gt; </h3>

It's your turn! Can you plot a histogram displaying the distribution of runners in ranges of completion hours? That is, how many runners completed the race in 3 hours? How many in 4 hours? So on and so forth. Before you plot the histogram, take a guess how long it took the most runners to complete the race? Between 2 - 3 hours? Between 3 - 4 hours? Then make the plot! Did you guess it right?

In [ ]:
### Prepare the data
# Reduce the dataframe to the column(s) of interest


In [ ]:
### Plot histogram

# Create the bins



# Make the histogram







# Create an interactive figure

Interactive charts can effectively tell a story. They also allow the audience to explore the information in a gradual and interactive way. Recall the interactive chart we played with in [DataViz 1](./data-viz1.ipynb) displaying the Ph.D. gender gap by country? When we look at all Ph.D. degrees regardless of the area of study, how well did we find the US does in terms of gender equity in awarding Ph.D. degrees? When we zoomed into "science and engineering", what did we find? When we zoomed into "social and behavioral", what did we find?    

The process of exploration is also a knowledge-building process. Interactive charts are really useful in helping the audience experience this process. Let's try building an interactive bar chart. 

## An interative bar chart

In [ ]:
# import needed modules
from ipywidgets import widgets, interactive

In [ ]:
# Get a general idea of the data
run.info()

In [ ]:
# Reduce the dataframe to data of interest
run_interactive = run[['Gender', 'CountryOfResName']].copy()

# Sort the Country names in alphabetical order for later use
run_interactive = run_interactive.sort_values(by='CountryOfResName').reset_index(drop=True)

# Drop the row where the CountryOfResName has a null value
run_interactive = run_interactive.dropna(subset='CountryOfResName').reset_index(drop=True)

In [ ]:
# Make a dropdown menu to select the Country of Residence, or "All"
country_res = widgets.Dropdown(
                                options=['All'] + run_interactive['CountryOfResName'].unique().tolist(),                            
                                value='All',
                                description='Country:',
                        )

In [ ]:
# Define a function to make the plot

def plot(country_res):
    df = run_interactive.copy()
    if country_res != 'All':
        df = df[df['CountryOfResName'] == country_res]
    x = ['Female', 'Male']
    y = [len(df.loc[df['Gender']=='F']), len(df.loc[df['Gender']=='M'])]
    fig, ax = plt.subplots()
    ax.bar(x, y, 0.2, color=['orchid', 'blue'])
    ax.set_ylabel('Number of runners')
    ax.set_title(f"Country of residence = {country_res}")

In [ ]:
# Render the plot
interactive(plot, country_res=country_res)

You have learned how to create scatter plots, line graphs, bar charts, and histograms. You can try and make them all interactive.

<h3 style="color:red; display:inline">Coding Challenge! &lt; / &gt; </h3>

It's your turn! Can you plot an interactive line graph showing the number of male and female runners in the Boston Marathon across 2017 - 2022 by Country of Residence? The plot will have three widgets:
* Start year
* End year
* Runner's country of residence

When a user chooses the start year, end year, and the country of residence, they will see a line graph with two lines showing the number of male and female runners over the selected year range for the selected country.

In [ ]:
### Create an interactive line graph
### based on the data on the Boston Marathon 
### between 2017 - 2022

# Get the paths to the files

paths = ['../All-sample-files/DataViz3_BostonMarathon2017.csv',
       '../All-sample-files/DataViz3_BostonMarathon2018.csv',
       '../All-sample-files/DataViz3_BostonMarathon2019.csv',
       '../All-sample-files/DataViz3_BostonMarathon2021.csv',
       '../All-sample-files/DataViz3_BostonMarathon2022.csv']

In [ ]:
### Prepare the data



In [ ]:
### Make the interactive chart



## Lesson Complete

Congratulations! You've completed the Data Visualization series.

Considering the amount of material in in this series, there's a good chance you won't retain it all. That's okay. Programmers often need to look up things to accomplish a task they haven't done in a while, particularly if it is in a language they don't often use. When you're working on a project, you can always come back to these lessons as reference materials. In other words, you've learned an incredible amount, so don't be surprised if it doesn't all stick at first.

If you want to help yourself retain what you've learned, the best way is to start putting it into practice. Try your hand at creating some small Data Visualization projects and recognize that the things you've learned here will cement with time and practice. When you do forget a particular thing—as we all do—a quick web search often turns up some useful examples.

### Solutions to exercises

Here are the solutions to some of the exercises in this notebook.

In [ ]:
### Make a figure containing multiple subplots
### showing the percentage of male and female children with elevated BLL by county

# prepare the data
lead = pd.read_csv('./data/lead_poisoning_county_2012To2020.csv')
lead['Rate'] = lead['Rate'] * 0.1
lead_by_gender = lead.loc[lead['Sex']!='Total', ['Geo Description', 'Sex', 'Year', 'Rate']]
# lead_female = lead.loc[lead['Sex']=='Female', ['Geo Description', 'Year', 'Rate']].copy().reset_index(drop=True)
# lead_male = lead.loc[lead['Sex']=='Male', ['Geo Description', 'Year', 'Rate']].copy().reset_index(drop=True)

# create the figure
import matplotlib.ticker as mtick
fig, axs = plt.subplots(nrows=7, ncols=2, figsize = (7, 15), sharex=True, sharey=True)

# Get the counties
counties = lead_by_gender['Geo Description'].unique().tolist()
labels=['Female', 'Male']
# Make the subplots
for i, ax in enumerate(axs.flat):
    ax.plot(range(2012, 2021), 
            lead_by_gender.loc[(lead_by_gender['Geo Description']==counties[i])&(lead_by_gender['Sex']=='Female'), 
                            'Rate'], 
            color='orange')
    ax.plot(range(2012, 2021), 
            lead_by_gender.loc[(lead_by_gender['Geo Description']==counties[i])&(lead_by_gender['Sex']=='Male'), 
                            'Rate'],             
            color='blue')
    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
    ax.set_title(counties[i], fontsize=9)
fig.suptitle('Percentage of Female and Male Children\nwith Elevated BLL', y=0.99)
fig.supxlabel('Year')
fig.legend(labels=labels,loc="upper right")
fig.tight_layout()

In [ ]:
### Create a histogram displaying the distribution of runners
### by completion hours

# Get the data
file ='../All-sample-files/DataViz3_BostonMarathon2022.csv'

# Create a dataframe from the csv file
run = pd.read_csv(file)

# Reduce the dataframe to the column of interest
run_hour = run[['OfficialTime']].copy()
run_hour = run_hour.rename(columns={'OfficialTime':'CompletionTime'})

# Get the hour
run_hour['CompletionTime'] = run_hour['CompletionTime'].str[0].astype(int)

# Plot histogram
bins = range(run_hour['CompletionTime'].min(), run_hour['CompletionTime'].max()+1)

# Plot the histagram
fig, ax = plt.subplots(figsize=(5, 5))
counts, edges, bars = ax.hist(run_hour['CompletionTime'], bins = bins, edgecolor='black')
ax.bar_label(bars);

In [ ]:
### Create an interactive line graph
### based on the data on the Boston Marathon 
### between 2017 - 2022
paths = ['../All-sample-files/DataViz3_BostonMarathon2017.csv',
       '../All-sample-files/DataViz3_BostonMarathon2018.csv',
       '../All-sample-files/DataViz3_BostonMarathon2019.csv',
       '../All-sample-files/DataViz3_BostonMarathon2021.csv',
       '../All-sample-files/DataViz3_BostonMarathon2022.csv']

# Get the csv files
BosRun_2017 = pd.read_csv(paths[0])
BosRun_2018 = pd.read_csv(paths[1])
BosRun_2019 = pd.read_csv(paths[2])
BosRun_2021 = pd.read_csv(paths[3])
BosRun_2022 = pd.read_csv(paths[4])

# reduce the columns to columns of interest
BosRun_2017 = BosRun_2017[['CountryOfResName', 'Gender']]
BosRun_2018 = BosRun_2018[['CountryOfResName', 'Gender']]
BosRun_2019 = BosRun_2019[['CountryOfResName', 'Gender']]
BosRun_2021 = BosRun_2021[['CountryOfResName', 'Gender']]
BosRun_2022 = BosRun_2022[['CountryOfResName', 'Gender']]

# Get the count of female and male runners for each year's dataframe
BosRun_2017 = BosRun_2017.groupby(['CountryOfResName', 'Gender']).size().to_frame('Count').stack().unstack(level=1)
BosRun_2017 = BosRun_2017.rename_axis(None, axis=1).reset_index().drop('level_1', axis=1).fillna(0)
BosRun_2018 = BosRun_2018.groupby(['CountryOfResName', 'Gender']).size().to_frame('Count').stack().unstack(level=1)
BosRun_2018 = BosRun_2018.rename_axis(None, axis=1).reset_index().drop('level_1', axis=1).fillna(0)
BosRun_2019 = BosRun_2019.groupby(['CountryOfResName', 'Gender']).size().to_frame('Count').stack().unstack(level=1)
BosRun_2019 = BosRun_2019.rename_axis(None, axis=1).reset_index().drop('level_1', axis=1).fillna(0)
BosRun_2021 = BosRun_2021.groupby(['CountryOfResName', 'Gender']).size().to_frame('Count').stack().unstack(level=1)
BosRun_2021 = BosRun_2021.rename_axis(None, axis=1).reset_index().drop('level_1', axis=1).fillna(0)
BosRun_2022 = BosRun_2022.groupby(['CountryOfResName', 'Gender']).size().to_frame('Count').stack().unstack(level=1)
BosRun_2022 = BosRun_2022.rename_axis(None, axis=1).reset_index().drop('level_1', axis=1).fillna(0)

# Add a new column to store the year of the race
BosRun_2017['Year'] = 2017
BosRun_2018['Year'] = 2018
BosRun_2019['Year'] = 2019
BosRun_2021['Year'] = 2021
BosRun_2022['Year'] = 2022

# Get all Country Res Names and make a dataframe
country_res_all = pd.DataFrame({'CountryOfResName':
                               list(set(BosRun_2017['CountryOfResName'].tolist() +
                                   BosRun_2018['CountryOfResName'].tolist() +
                                   BosRun_2019['CountryOfResName'].tolist() +
                                   BosRun_2021['CountryOfResName'].tolist() +
                                   BosRun_2022['CountryOfResName'].tolist()
                                  ))}).sort_values(by='CountryOfResName').reset_index(drop=True)

# Make df from each year a full df with all country names
# fill the missing values with relevant values
BosRun_2017 = pd.merge(BosRun_2017, country_res_all, how='right').fillna(value={'F':0, 'M':0, 'Year':2017})
BosRun_2018 = pd.merge(BosRun_2018, country_res_all, how='right').fillna(value={'F':0, 'M':0, 'Year':2018})
BosRun_2019 = pd.merge(BosRun_2019, country_res_all, how='right').fillna(value={'F':0, 'M':0, 'Year':2019})
BosRun_2021 = pd.merge(BosRun_2021, country_res_all, how='right').fillna(value={'F':0, 'M':0, 'Year':2021})
BosRun_2022 = pd.merge(BosRun_2022, country_res_all, how='right').fillna(value={'F':0, 'M':0, 'Year':2022})

# Concatenate the dfs into one single df
BosRun_all = pd.concat([BosRun_2017, BosRun_2018, BosRun_2019, BosRun_2021, BosRun_2022])

# Drop rows where the CountryOfResName has a null value
BosRun_all = BosRun_all.dropna(subset='CountryOfResName')

# Make two bounded text box for year range selection 
start_year = widgets.BoundedFloatText(
    value=BosRun_all['Year'].min(),
    min=BosRun_all['Year'].min(),
    max=BosRun_all['Year'].max(),
    step=1,
    description='Start Year:',
    disabled=False,
    color='black'
)
end_year = widgets.BoundedFloatText(
    value=BosRun_all['Year'].max(),
    min=BosRun_all['Year'].min(),
    max=BosRun_all['Year'].max(),
    step=1,
    description='End Year:',
    disabled=False,
    color='black'
)

# make dropdown menu for country selection 
country_res = widgets.Dropdown(
                                options=['All'] + BosRun_all['CountryOfResName'].unique().tolist(),                            
                                value='All',
                                description='Country:',
                        )

# Define a function to plot the interactive figure
def plot(start_year, end_year, country_res):
    """
    Filter and plot the dataframe as a line graph of count of F and M runners over the
    selected year range
    """
    if start_year >= end_year:
        print("Start year must preceed end year.")        
    else:
        df = BosRun_all.copy()
        df = df.loc[(df['Year'] >= start_year) & (df['Year'] <= end_year)]
        x = list(range(int(start_year), int(end_year+1)))
        if 2020 in x:
            x.remove(2020)
        if country_res != 'All':
            df = df.loc[df['CountryOfResName']==country_res]
        # Plot the graph

        if df['F'].sum() + df['M'].sum() == 0:
            print('There is no data in the selection to plot.')
        else:
            y_female = df[['F', 'M', 'Year']].groupby(by='Year').sum()['F']
            y_male = df[['F', 'M', 'Year']].groupby(by='Year').sum()['M']
            fig, ax = plt.subplots()
            ax.plot(x, y_female, 'o-', color='orchid', label='F')
            ax.plot(x, y_male, 'D-', color='blue', label='M')
            ax.set_xticks(x)
            ax.legend(loc='lower left')
            ax.set_title(f"Country of residence = {country_res}")
            plt.show()
interactive(plot, country_res=country_res, start_year=start_year, end_year=end_year)